https://gis.stackexchange.com/questions/350771/earth-engine-simplest-way-to-move-from-ee-image-to-array-for-use-in-sklearn

In [1]:
cd

/home/jovyan


### Setup

#### Install GEE API

In [2]:
# !pip install earthengine-api

In [3]:
!pip install -U earthengine-api

     |████████████████████████████████| 157 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 92 kB 6.0 MB/s  eta 0:00:01
     |████████████████████████████████| 61 kB 14.8 MB/s eta 0:00:01
  Using cached google_auth_httplib2-0.0.4-py2.py3-none-any.whl (9.1 kB)
  Using cached httplib2-0.18.1-py3-none-any.whl (95 kB)
Processing ./.cache/pip/wheels/50/6c/00/2a8868de404e13a77667ff2c94e908e7cff9acd1d1dd307a48/httplib2shim-0.0.3-py2.py3-none-any.whl
     |████████████████████████████████| 75 kB 8.3 MB/s  eta 0:00:01
     |████████████████████████████████| 91 kB 9.1 MB/s  eta 0:00:01
  Using cached uritemplate-3.0.1-py2.py3-none-any.whl (15 kB)
  Using cached google_crc32c-1.0.0-cp36-cp36m-manylinux2010_x86_64.whl (39 kB)
  Using cached googleapis_common_protos-1.52.0-py2.py3-none-any.whl (100 kB)
  Created wheel for earthengine-api: filename=earthengine_api-0.1.241-py3-none-any.whl size=185020 sha256=29bc568768755e8153d2dbc8f72614505259c32982ab8e961c84807a9313711a
  Stored in

#### Import modules

In [6]:
import ee
import numpy as np
import matplotlib.pyplot as plt
import sys
import folium
import datacube

sys.path.append("Scripts")
from dea_plotting import display_map

In [7]:
dc = datacube.Datacube(app='WCF')

#### Authenticate

In [8]:
ee.Authenticate()

Enter verification code:  4/1AY0e-g4ldvi1mlgRPXHp_y28SaED4cJ7lkIDtVwdN8A2KknHsyjgfT6FDXk


URLError: <urlopen error [Errno 99] Cannot assign requested address>

In [ ]:
ee.Initialize()

### Define parameters 

In [ ]:
# Define parameters

# Dates
# NOTE: GEE dates are inclusive and GA are exclusive on the end date
start = '2018-01-01'
end_ga = '2018-12-31'
end_gee = '2019-01-01'

# Set the central latitude and longitude
central_lat = -35.555439 
central_lon = 138.886889

In [ ]:
# Set the buffer to load around the central coordinates
buffer = 0.05 # must be small for .sampleRectangle() to work

# Compute the bounding box for the study area
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

display_map(x=study_area_lon, y=study_area_lat, margin=-0.2)

### Load GEE imagery

In [ ]:
# Define an area of interest.
aoi = ee.Geometry.Polygon(
  [[[study_area_lon[0], study_area_lat[0]],
    [study_area_lon[0], study_area_lat[1]],
    [study_area_lon[1], study_area_lat[1]],
    [study_area_lon[1], study_area_lat[0]]]], None, False)

# Filter collection to get desired image
img = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
        .filterBounds(aoi)\
        .filterDate(start,end_gee)\
        .filter(ee.Filter.lt('CLOUD_COVER',5))\
        .select(['B4', 'B3', 'B2'])\
        .first()\

### Display GEE image in folium

Define parameters for display

In [ ]:
# Map location in lats and lons
location=[central_lat,central_lon]
# How zoomed in the map is
zoom_start=10

# Image to use
image = img
# Set visualization parameters.
vis_params = {
  'bands':['B4','B3','B2'], # check which abnds here https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#bands
  'min': 0, # pixel value
  'max': 3000 # pixel value
}
display_name = 'Sentinel 2 median'

Create custom function to display GEE image in folium map

In [ ]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


Display your map

In [ ]:
# Create a folium map object.
my_map = folium.Map(location=location, zoom_start=zoom_start, height=500)

# Add the elevation model to the map object.
my_map.add_ee_layer(img, vis_params, display_name)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

### Convert image into a numpy array

In [ ]:
img

In [ ]:
# Get 2-d pixel array for AOI - returns feature with 2-D pixel array as property per band.
band_arrs = img.sampleRectangle(region=aoi)

# Get individual band arrays.
band_arr_b4 = band_arrs.get('B2') #B2
band_arr_b5 = band_arrs.get('B3') #B3
band_arr_b6 = band_arrs.get('B4') #B4

# Transfer the arrays from server to client and cast as np array.
np_arr_b4 = np.array(band_arr_b4.getInfo())
np_arr_b5 = np.array(band_arr_b5.getInfo())
np_arr_b6 = np.array(band_arr_b6.getInfo())
print(np_arr_b4.shape)
print(np_arr_b5.shape)
print(np_arr_b6.shape)

# Expand the dimensions of the images so they can be concatenated into 3-D.
np_arr_b4 = np.expand_dims(np_arr_b4, 2)
np_arr_b5 = np.expand_dims(np_arr_b5, 2)
np_arr_b6 = np.expand_dims(np_arr_b6, 2)
print(np_arr_b4.shape)
print(np_arr_b5.shape)
print(np_arr_b6.shape)

# Stack the individual bands to make a 3-D array.
rgb_img = np.concatenate((np_arr_b6, np_arr_b5, np_arr_b4), 2)
print(rgb_img.shape)

### Display new numpy array

In [ ]:
# Scale the data to [0, 255] to show as an RGB image.
rgb_img_test = (255*((rgb_img - 100)/3000)).astype('uint8')
plt.imshow(rgb_img_test)
plt.show()

In [ ]:
# Convert numpy array to xarray
rgb_img_test.shape

In [ ]:
plt.imshow(rgb_img_test[:,:,0])
plt.show()

### Load GA data

In [ ]:
#Load all data in baseline period available from Near Real Time data

wofs = dc.load(
              product=['wofs_annual_summary'],
              x= study_area_lon,
              y= study_area_lat,
              time=(start, end_ga),
              #measurements=['nbar_blue','nbar_green','nbar_red','nbar_nir_1', 'nbar_swir_3'],
              output_crs='EPSG:4326',
              crs='EPSG:4326',
              resolution=(-30, 30),
              #group_by='solar_day'
)

In [ ]:
wofs

In [ ]:
wofs.frequency.plot()

In [ ]:
import xarray as xr

xr.DataArray(rgb_img_test,dims=())

In [ ]:
wofs.squeeze(dim='time').coords

In [ ]:
wofs.coords['y']

In [ ]:
wofs.coords['x']

In [ ]:
rgb_img_test[:,:,0].shape

In [ ]:
xr.DataArray(np_arr_b4[:,:,0],coords=wofs.squeeze(dim='time').coords)